In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
import pandas as pd

In [2]:
# init model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
# init softmax to get probabilities later on
sm = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def get_probs(sentence, nouns):
    
    token_ids = tokenizer.encode(sentence, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

    # forward
    output = model(token_ids)
    last_hidden_state = output[0].squeeze(0)
    # only get output for masked token
    # output is the size of the vocabulary
    mask_hidden_state = last_hidden_state[masked_position]
    # convert to probabilities (softmax)
    # giving a probability for each item in the vocabulary
    probs = sm(mask_hidden_state)
    
    return([probs[tokenizer.convert_tokens_to_ids(noun)].item() for noun in nouns])

In [12]:
text_file = open("nounlist.txt", "r")
nouns = text_file.read().splitlines() 

a_sentence = f"A {tokenizer.mask_token} is an animal"
an_sentence = f"An {tokenizer.mask_token} is an animal"

token_is_a_animal_probs = get_probs(a_sentence,nouns)
token_is_an_animal_probs = get_probs(an_sentence,nouns)

animal_probs = [max(token_is_an_animal_probs[i],token_is_a_animal_probs[i]) for i,x in enumerate(token_is_an_animal_probs)]

animal_df = pd.DataFrame({'nouns':nouns, 'animal':animal_probs}).sort_values(by=['animal'],ascending=False)
animal_df.loc[animal_df['nouns']=='cat']
animal_df.to_csv('animal_probs.csv')

In [16]:
a_fruit_sentence = f"A {tokenizer.mask_token} is a fruit"
an_fruit_sentence = f"An {tokenizer.mask_token} is a fruit"

token_is_a_fruit_probs = get_probs(a_fruit_sentence,nouns)
token_is_an_fruit_probs = get_probs(an_fruit_sentence,nouns)

fruit_probs = [max(token_is_an_fruit_probs[i],token_is_an_fruit_probs[i]) for i,x in enumerate(token_is_an_fruit_probs)]

fruit_df = pd.DataFrame({'nouns':nouns, 'fruit':fruit_probs}).sort_values(by=['fruit'],ascending=False)
fruit_df.to_csv('fruit_probs.csv')

In [28]:
nouns = ['dog','spider']

In [29]:
animal_probs = [tokenizer.convert_tokens_to_ids(noun) for noun in nouns]

In [30]:
animal_probs

[3899, 6804]

In [38]:
probs[pudding_id]

tensor(5.5253e-06)